<a href="https://colab.research.google.com/github/prasann25/colab/blob/main/04_transfer_learning_in_tensorflow-part1_feature_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

001